# VecOps
This tutorial shows how VecOps can be used to slim down the programming model typically adopted in HEP for analysis.
We compare the approach of PyROOT and of RDataFrame.

Our dataset contains for every event three collections of floating point numbers of equal size: **E**, **px** and **py**. We can imagine these are properties of muons, or more in general, of *candidates*.
Our goal is to:.
1. Select the candidates in all events for which the energy, *E*, is greater than 100.
2. Calculate the transverse momentum as the square sum of **px** and **py**.
3. Fill a histogram with the transverse momenta.

In [1]:
import ROOT
from math import sqrt

filename = "vecOpsHEP.root"
treename = "myDataset"
RDF = ROOT.ROOT.RDataFrame

Welcome to JupyROOT 6.15/01


Here we show how to perform an event loop with PyROOT: it's quite nice, but we can do better, both in terms of performance and programming model

In [2]:
def WithPyROOT():
    f = ROOT.TFile(filename)
    h = ROOT.TH1F("pt", "pt", 16, 0, 4)
    for event in f.myDataset:
        for E, px, py in zip(event.E, event.px, event.py):
            if (E > 100):
               h.Fill(sqrt(px*px + py*py))
    h.DrawCopy()

This is how the same operations are performed with a RDataFrame. The amount of code is less and density of meaning increases. In addition, parallelisation and vectorisation kick in without much work being required from the user.

In [3]:

ROOT.ROOT.EnableImplicitMT()
def WithRDataFrameVecOpsJit():
    f = RDF(treename, filename)
    h = f.Define("good_pt", "sqrt(px*px + py*py)[E>100]")\
         .Histo1D(("pt", "pt", 16, 0, 4), "good_pt")
    h.DrawCopy()

 We plot twice the same quantity, the key is to look into the implementation
 of the functions above

In [4]:
c = ROOT.TCanvas()
c.Divide(2,1)
c.cd(1)
WithPyROOT()
c.cd(2)
WithRDataFrameVecOpsJit()

Draw all canvases 

In [6]:
%jsroot on
c.Draw()